In [1]:
import os
import sys
sys.path.append("..")

import tensorflow as tf

import cst_model as cst
import distortion_layers as ly
import base_models as bm
import callbacks as cb


# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) 

In [2]:
os.getcwd()

'/main_dir/felipe/projects/CST/notebooks'

### Parameters


In [4]:
# train_path = "../data"
train_path = "/main_dir/felipe/projects/cp_toolbox_data/artifacts_project/training_data/v4"

tile_size = 128
batch_size = 32
channels = 3
# n_st_components = 2
alpha = 2
epochs = 10
class_mode = "binary"

validation_split = 0.2

### Create distortion layer

In [5]:
dcs = {
    "contrast": {"factor": [0.2, 0.2]},
    "color": {"factor": [20,0,20]},
    "blur": {"filter_shape": 2, "sigma": 5.},  # kernel size is 'filter_shape * 2 + 1'
    "brightness": {"lower": .85, "upper":1.15}
}

layers = [
    ly.RandomColorByChannel(**dcs["color"]), 
    tf.keras.layers.RandomContrast(**dcs["contrast"]),
    ly.RandomBrightness(**dcs["brightness"]),
    ly.RandomGaussianBlur(**dcs["blur"]),
    ly.BlueRedChannelSwapLayer(),
]


# dist_layer = [AdjSaturation(10),
#               tf.keras.layers.Lambda(lambda x: x * 0. + 55)]

dist_layer = layers



2023-07-05 16:32:10.465400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-05 16:32:10.476803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-05 16:32:10.477003: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-05 16:32:10.478180: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Load images and assign parameters

In [6]:
gen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = validation_split
)

t_flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode='rgb',  # rgb for color
    batch_size=batch_size,
    class_mode=class_mode,  # 'sparse' for multiclass, 'binary' for binary 
    subset='training'
)

v_flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode="rgb",  # rgb for color
    batch_size=batch_size,
    shuffle=False,
    class_mode=class_mode,  # 'sparse' for multiclass, 'binary' for binary
    subset='validation'
)

Found 608896 images belonging to 4 classes.
Found 152224 images belonging to 4 classes.


### Load and compile network

In [7]:
if class_mode == "binary":
    metrics = [
        tf.keras.metrics.BinaryCrossentropy(name="bce"),  # BinaryCrossentropy for binary
        tf.keras.metrics.BinaryAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.binary_crossentropy
    final_layer_node = 1
    binary = True
    
elif class_mode == "sparse":
    metrics = [
        tf.keras.metrics.SparseCategoricalCrossentropy(name="sce"), 
        tf.keras.metrics.SparseCategoricalAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.sparse_categorical_crossentropy
    final_layer_node = t_flow.num_classes
    binary = False

elif class_mode == "categorical":
    metrics = [
        tf.keras.metrics.CategoricalCrossentropy(name="cce"),
        tf.keras.metrics.CategoricalAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.categorical_crossentropy
    final_layer_node = t_flow.num_classes
    binary = False
    
else:
    print("no class mode provided")
    
    
base_model = bm.create_thesis_model(tile_size=tile_size, channels=3, final_layer_node=final_layer_node)
i = tf.keras.Input(shape=(tile_size, tile_size, channels))
x_i = base_model(i)
cst_model = cst.CSTModel(inputs=i, outputs=x_i, alpha=alpha, n_st_components=n_st_components, 
                         dist_layers=dist_layer, binary=binary)


cst_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True),
    loss = base_loss,
    metrics = metrics 
)


### Train network

In [8]:

cst_model.run_eagerly = False

cst_model.fit(
    x=t_flow,
    validation_data=v_flow,
    epochs=epochs,
    class_weight=bm.get_class_weights(t_flow.classes)
)

Epoch 1/10


2023-07-05 16:32:38.320830: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2023-07-05 16:32:39.296516: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


19028/19028 [==============================] - 5261s 276ms/step - loss_0: 0.2663 - cst_metric: 0.3657 - sce_: 0.3069 - acc_: 0.8875 - sce_0: 0.3198 - acc_0: 0.8818 - sce_1: 0.3071 - acc_1: 0.8876 - sce_2: 0.3155 - acc_2: 0.8840 - sce_3: 0.3242 - acc_3: 0.8792 - sce_4: 0.3196 - acc_4: 0.8800 - val_sce: 0.1989 - val_acc: 0.9302
Epoch 2/10
19028/19028 [==============================] - 5245s 276ms/step - loss_0: 0.1126 - cst_metric: 0.1217 - sce_: 0.1874 - acc_: 0.9348 - sce_0: 0.1976 - acc_0: 0.9301 - sce_1: 0.1863 - acc_1: 0.9347 - sce_2: 0.1939 - acc_2: 0.9317 - sce_3: 0.2043 - acc_3: 0.9271 - sce_4: 0.1948 - acc_4: 0.9306 - val_sce: 0.1719 - val_acc: 0.9405
Epoch 3/10
19028/19028 [==============================] - 5238s 275ms/step - loss_0: 0.0912 - cst_metric: 0.0971 - sce_: 0.1651 - acc_: 0.9430 - sce_0: 0.1631 - acc_0: 0.9426 - sce_1: 0.1535 - acc_1: 0.9466 - sce_2: 0.1605 - acc_2: 0.9438 - sce_3: 0.1710 - acc_3: 0.9392 - sce_4: 0.1605 - acc_4: 0.9432 - val_sce: 0.1560 - val_acc: 0